In [4]:
# https://drlee.io/advanced-stock-pattern-prediction-using-lstm-with-the-attention-mechanism-in-tensorflow-a-step-by-143a2e8b0e95

from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard, CSVLogger

import os
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from tensorflow.python.client import device_lib 
# Set random seed for reproducibility
tf.random.set_seed(42)
keras.utils.set_random_seed(42)

import sys

sys.path.append("src/")
from model.evaluation import create_results_df
from model.tf_models import create_model_LSTM_with_Attention
from data.preparation import load_dataset,prepare_data 


# Configuration
DATA_DIR = 'data/'
PATH_REPORTS = 'reports/'
PATH_MODELS = 'models/'
PATH_LOGS = "logs"

ASSETS = ["PETR3.SA", "PRIO3.SA", "VALE3.SA", "GGBR3.SA", "ABCB4.SA", "ITUB3.SA", "FLRY3.SA", "RADL3.SA"]
SEQ_LEN = 60
FEATURES_COLS = ['diff_close_mean_z_score']
LABEL_COL = ['diff_close_mean_z_score']

   
print(device_lib.list_local_devices())
print("TensorFlow Version: ", tf.__version__)

for asset in ASSETS:
    dataset = load_dataset(asset, DATA_DIR)
    X_train, X_test, y_train, y_test = prepare_data(dataset, SEQ_LEN, FEATURES_COLS, LABEL_COL)
    
    model = create_model_LSTM_with_Attention((X_train.shape[1], X_train.shape[2]))
    print(model.summary())

    callbacks = [
        EarlyStopping(monitor='val_loss', patience=10),
        ModelCheckpoint(PATH_MODELS + f'best_model_LSTM_with_Attention_{asset.replace(".", "_")}.keras', save_best_only=True, monitor='val_loss'),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5),
        TensorBoard(log_dir=PATH_LOGS),
        CSVLogger(PATH_MODELS + f'training_log_LSTM_with_Attention_{asset.replace(".", "_")}.csv')
    ]

    model.fit(X_train, y_train, epochs=100, batch_size=25, validation_split=0.2, callbacks=callbacks)
    
    y_pred = model.predict(X_test)
    
    y_test = list(y_test.reshape(-1))
    y_pred = list(y_pred.reshape(y_pred.shape[0]))
    
    results_df = create_results_df(y_test, y_pred)
    
    results_df.to_csv(PATH_REPORTS + f'test_results/LSTM_with_Attention_{asset.replace(".", "_")}_features={"_".join(FEATURES_COLS)}__label={"_".join(LABEL_COL)}_test_results.csv', index = False)


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17216993194691612750
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4919787520
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11267290509001246039
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1660 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]
TensorFlow Version:  2.17.0


I0000 00:00:1722391907.414400   52932 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1722391907.414826   52932 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1722391907.415067   52932 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1722391907.415335   52932 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 60, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ (None, 60, 50)    │     10,400 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ (None, 60, 50)    │     20,200 │ lstm_2[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_weight    │ (None, 60, 50)    │         50 │ lstm_3[0][0],     │
│ (AdditiveAttention) │                   │            │ lstm_3[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_1          │ (None, 60, 50)    │          0 │ lstm_3[0][0],     │
│ (Multiply)          │                   │            │ attention_weight… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 3000)      │          0 │ multiply_1[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │      3,001 │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 1)         │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 1)         │          4 │ dropout_1[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 33,655 (131.46 KB)

 Trainable params: 33,653 (131.46 KB)

 Non-trainable params: 2 (8.00 B)

None
Epoch 1/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 12s 64ms/step - cross entropy: -5.9956 - loss: 1.3539 - mape: 204.6299 - mse: 0.9460 - r2_score: 0.4487 - val_cross entropy: -4.7339 - val_loss: 1.5341 - val_mape: 102.5383 - val_mse: 1.5341 - val_r2_score: 0.0920 - learning_rate: 0.0010
Epoch 2/100
113/122 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - cross entropy: -7.3463 - loss: 0.8399 - mape: 149.2874 - mse: 0.8399 - r2_score: 0.5106

KeyboardInterrupt: 